# D-Waveのハイブリッドソルバーを，amplify上で動かす

## 前準備

- 仮想環境を作成する．このとき，Pythonのバージョンは3.9以上．pipは19.3以上であることを確認．
- 仮想環境上で，``pip install amplify``を実行．さらに``pip install amplify[extra]``を実行．(conda環境でもpip) 
    - [こちらのページ](https://amplify.fixstars.com/ja/docs/amplify/v0/quickstart.html)	を参考にするとよい．

## 以下のサンプルプログラムの問題設定
$H=-x_0x_2-x_1+x_2$の最小値を求める．最小値は$H=-1$であり，その際の変数は$x_0=1, x_1=1, x_2=1$などが考えられる．(複数あり)


## 参考URL
- [D-Waveの使い方(Fixstars公式)](https://amplify.fixstars.com/ja/docs/amplify/v1/clients/dwave.html)
- [係数行列を使う方法(Fixstars公式)](https://amplify.fixstars.com/ja/docs/amplify/v1/matrix.html)
- [部分和問題のサンプルプログラム(Fixstars公式)](https://amplify.fixstars.com/ja/docs/amplify/v1/subset_sum.html)

In [1]:
from amplify import LeapHybridSamplerClient
from amplify import Model, VariableGenerator, solve

In [2]:
client = LeapHybridSamplerClient()
client.token = "DWAVE/J6kxAdgJaVf4GnGuDoa5jZkdQJPuvaXr" # トークン
client.solver = "hybrid_binary_quadratic_model_version2p" # ソルバー
client.parameters.time_limit = 3 # 実行時間

In [3]:
import numpy
from amplify import VariableGenerator

gen = VariableGenerator()
q = gen.matrix("Binary", 3) # 変数の数を指定．
q.quadratic = numpy.array([[0, 0, -1],[0, -1, 0], [0, 0, 1]]) # 係数行列 Liner部分も別途設定できるが，おそらくこれで十分．
model = Model(q)
print(model)

minimize:
  (x^T) Q x + (p^T) x + c
  where:
    x = [q_0, q_1, q_2],
    Q = [[ 0.,  0., -1.],
         [ 0., -1.,  0.],
         [ 0.,  0.,  1.]],
    p = [ 0.,  0.,  0.],
    c = 0


In [4]:
# 実行
result = solve(model, client)

In [5]:
# 最小値
q_values = q.evaluate(result.best.values)

print(q_values)

-1.0


In [6]:
# 最小値になるときの値．
result_array = result.best.values

In [7]:
result_dict = dict(result_array)

In [8]:
ADic_val = list(result_dict.values())
ADic_val

[1.0, 1.0, 0.0]

## 注意点

今回はハイブリッドソルバーだが，QA単体の場合は，別途プログラムを書き直す必要がある．その際は[こちら](https://amplify.fixstars.com/ja/docs/amplify/v1/clients/dwave.html)を参考にするとよい．

おそらく，fixstars経由でAdvantageなどを使う際には，解の個数(num_reads)がD-Wave Ocean同様に設定できる．そのため，num_readsの設定と，上記のq_valueやresult_arrayの型が変わってくると考えられる．